load data

In [23]:
# pip install gensim

In [24]:
import pandas as pd 
import numpy as np 
from gensim.models import FastText
import json

In [9]:
data=pd.read_csv(r"preprocessed_data.csv")
data.head()

,Unnamed: 0,content,labels
0,0,{'tamil_sentences': ['தென்காசி தொகுதி புதிய தம...,0
1,1,{'tamil_sentences': ['அண்ணன் இதனை சூசகமாக மாதங...,1
2,2,{'tamil_sentences': ['ஒரு வருடம் ஆகி விட்டது இ...,2
3,3,{'tamil_sentences': ['எடப்பாடியை கண்டுகொள்ளாத ...,5
4,4,{'tamil_sentences': ['எங்களின் அரசியல் அடுத்த ...,2


preparing data for fasttext embedding model


In [10]:
data['content'][1]

"{'tamil_sentences': ['அண்ணன் இதனை சூசகமாக மாதங்கள் முன்பே பேட்டியில் சொல்லிட்டார்'], 'emojis': [], 'tags': [], 'tokens': ['அண்ணன்', 'இதனை', 'சூசகமாக', 'மாதங்கள்', 'முன்பே', 'பேட்டியில்', 'சொல்லிட்டார்'], 'lemmas': ['அண்ணன்', 'இது', 'சூசகம்', 'மாதம்', 'முன்பே', 'பேட்டி', 'சொல்லிடு'], 'pos': ['PROPN', 'PRON', 'ADV', 'NOUN', 'ADP', 'NOUN', 'VERB'], 'dependancy_relation': ['nmod', 'obj', 'advmod', 'obl', 'case', 'obl', 'root']}"

In [11]:
size=10

In [71]:
text_list=[]
labels=[]
with open('sentences.txt','w',encoding='utf-8') as f:
        for i in range(len(data)):
            try:
                content=data['content'][i]
                content=content.replace("'",'"')
                content=json.loads(f"{content}")
                text=content['tamil_sentences'][0]+ ' '+' '.join(content['emojis']) +' '+ ' '.join(content['tags'])
                text_list.append(text.split())
                f.writelines(f'{text}\n')
                labels.append(data['labels'][i])
            except:
                print(1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [72]:
text_list[3:4]

[['எடப்பாடியை',
  'கண்டுகொள்ளாத',
  'எடப்பாடி',
  'ஆதரிப்பீர்',
  'உதயசூரியன்',
  'மக்களின்குரல்',
  'ஓமலூர்',
  'சேலம்',
  'சேலம்',
  'செல்வகணபதி',
  'சேலம்தெற்கு',
  'சேலம்வடக்கு',
  'எடப்பாடி',
  'சேலம்மேற்கு',
  '🫢',
  '🌄',
  '🏴',
  '🚩',
  '#மக்களின்குரல்',
  '#ஓமலூர்',
  '#சேலம்',
  '#சேலம்4செல்வகணபதி',
  '#Vote4DMK',
  '#Vote4INDIA',
  '#Testimonials',
  '#சேலம்தெற்கு',
  '#சேலம்வடக்கு',
  '#எடப்பாடி',
  '#சேலம்மேற்கு',
  '#Eps',
  '#Edappadi',
  '#Palanisamy']]

In [73]:
model=FastText(text_list,vector_size=240)

In [74]:
sentence_embeddings=[]

In [68]:
target_word="நாம்"
context_words =model.wv.most_similar(target_word, topn=10)
for word, similarity in context_words:
    print(f"{word}:{similarity:.4f}")

நாம்தமிழர்:0.9999
நாம்தமிழர்கட்சி:0.9999
#சீமானின்_சின்னம்_ஒலிவாங்கி_மைக்:0.9999
#சீமானின்_சின்னம்_ஒலிவாங்கி:0.9999
#நாம்தமிழர்கட்சி:0.9999
#மக்களின்_சின்னம்_ஒலிவாங்கி:0.9999
#Election2024:0.9999
#மக்களின்_சின்னம்_மைக்:0.9999
#Elections2024:0.9999
#நாம்தமிழர்:0.9999


In [76]:
len(labels)

4337

In [75]:
with open('sentences.txt','r',encoding='utf-8') as f:
    lines=f.readlines()
len(lines)

4337

In [82]:
from gensim.models import FastText

for sentence in lines:
    tokens = sentence.strip().split()  
    token_embeddings = [model.wv[token] for token in tokens if token in model.wv]

    if token_embeddings:
        sentence_embedding = sum(token_embeddings) / len(token_embeddings)
    else:
        sentence_embedding = None 
        print(1)
    sentence_embeddings.append(sentence_embedding)



In [83]:
len(sentence_embeddings)

4337

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [85]:
model=DecisionTreeClassifier()
x_train,x_test,y_train,y_test=train_test_split(sentence_embeddings,labels)

In [86]:
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [87]:
pred=model.predict(x_test)
acc=accuracy_score(pred,y_test)
print(acc)

0.23317972350230415
